In [50]:
import os
import pywt
import math
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
from matplotlib import cm
from scipy import stats
import numpy.polynomial.polynomial as poly

In [4]:
def import_file(path, limit_low=None, limit_high=None):

    spectrum = np.genfromtxt(path, delimiter=",")
    spectrum = np.transpose(spectrum)
    wavenumbers = spectrum[0]
    intensities = spectrum[1]

    if limit_low is not None:
        limit_low_index = list(wavenumbers).index(limit_low)
    else:
        limit_low_index = 0
        limit_low = wavenumbers[0]

    if limit_high is not None:
        limit_high_index = list(wavenumbers).index(limit_high)
    else:
        limit_high_index = len(wavenumbers)
        limit_high = wavenumbers[-1]

    wavenumbers = wavenumbers[limit_low_index:limit_high_index]
    intensities = intensities[limit_low_index:limit_high_index]
    return wavenumbers, intensities

def import_directory(path, limit_low=None, limit_high=None):
    # files = os.listdir(path)

    # for filename in files:
    #     np.genfromtxt(filename, delimiter=",")
    pass

In [5]:
wavenumbers, intensities = import_file("spectra/E (1).TXT", 300, 1600)

In [57]:
intensities_sg1 = sig.savgol_filter(intensities, 
                                    window_length=25, 
                                    polyorder=3, 
                                    deriv=1)

In [58]:
intensities_sg2 = sig.savgol_filter(intensities, 
                                    window_length=25, 
                                    polyorder=3, 
                                    deriv=2)

In [59]:
intensities_sg3 = sig.savgol_filter(intensities, 
                                    window_length=25, 
                                    polyorder=3, 
                                    deriv=3)

In [64]:
zeros_sg1 = np.diff(np.sign(intensities_sg1))
mins = np.where(zeros_sg1 > 0)[0]

In [61]:
zeros_sg3 = np.diff(np.sign(intensities_sg3))

In [62]:
peaks = np.where(zeros_sg3 > 0)[0]
peaks = peaks[intensities_sg2[peaks] < -threshold]

In [63]:
peak_condensing = []
peaks_new = []
for i in range(len(wavenumbers)-1):
    if i in peaks:
        peak_condensing.append(i)
    if intensities_sg2[i] > 0 and len(peak_condensing) > 0:
        peaks_new.append(int(np.mean(peak_condensing)))
        peak_condensing = []
if len(peak_condensing) > 0:
    peaks_new.append(int(np.mean(peak_condensing)))
peaks_new = np.asarray(peaks_new)            
print(len(peaks_new))

36


In [65]:
def plot_raw_spectra(wavenumbers, intensities0, intensities1, intensities2, intensities3):
    fig, ax = plt.subplots(2,2)
    ax[0,0].plot(wavenumbers, intensities0, linewidth = 1)
    ax[0,1].plot(wavenumbers, intensities1, linewidth = 1)
    ax[1,0].plot(wavenumbers, intensities2, linewidth = 1)
    ax[1,1].plot(wavenumbers, intensities3, linewidth = 1)
    major_ticks = ticker.MultipleLocator(100)
    minor_ticks = ticker.MultipleLocator(20)
    for i, plot in np.ndenumerate(ax):
        print(i)
        ax[i].set_xlim(wavenumbers[0], wavenumbers[-1])
        ax[i].invert_xaxis()
        ax[i].xaxis.set_major_locator(major_ticks)
        ax[i].xaxis.set_minor_locator(minor_ticks)
        ax[i].grid()
        if i != 0:
            ax[i].hlines(0, wavenumbers[0], wavenumbers[-1], linewidth=1, color="black")
    #ax[0,0].vlines(wavenumbers[peaks_new], 0, np.max(intensities)*1.1, linewidth=0.7, color="black")
    ax[0,0].vlines(wavenumbers[mins], 0, np.max(intensities)*1.1, linewidth=0.7, color="black")
    ax[0,0].set_ylim(np.min(intensities)*0.9, np.max(intensities)*1.1)

In [66]:
%matplotlib
plot_raw_spectra(wavenumbers, intensities, intensities_sg1, intensities_sg2, intensities_sg3)

Using matplotlib backend: Qt5Agg
(0, 0)
(0, 1)
(1, 0)
(1, 1)


In [51]:
?plt.xticks

Signature: plt.xticks(ticks=None, labels=None, **kwargs)
Docstring:
Get or set the current tick locations and labels of the x-axis.

Pass no arguments to return the current values without modifying them.

Parameters
----------
ticks : array-like, optional
    The list of xtick locations.  Passing an empty list removes all xticks.
labels : array-like, optional
    The labels to place at the given *ticks* locations.  This argument can
    only be passed if *ticks* is passed as well.
**kwargs
    `.Text` properties can be used to control the appearance of the labels.

Returns
-------
locs
    The list of xtick locations.
labels
    The list of xlabel `.Text` objects.

Notes
-----
Calling this function with no arguments (e.g. ``xticks()``) is the pyplot
equivalent of calling `~.Axes.get_xticks` and `~.Axes.get_xticklabels` on
the current axes.
Calling this function with arguments is the pyplot equivalent of calling
`~.Axes.set_xticks` and `~.Axes.set_xticklabels` on the current axes.

Exam